In [0]:
%sql 
use lakehouse

In [0]:
from pyspark.sql.types import *
df = spark.read.format("delta").table("Silver").select("date_listed")

In [0]:
from pyspark.sql.functions import to_date, year, month, dayofmonth

# Tạo các cột mới "year", "month", "day" từ cột "date_listed"
df = df.withColumn('year', year(to_date(df['date_listed'])))
df = df.withColumn('month', month(to_date(df['date_listed'])))
df = df.withColumn('day', dayofmonth(to_date(df['date_listed'])))
df  = df.select(["day","month","year","date_listed"])


In [0]:
df = df.dropDuplicates(["day","month","year"])


In [0]:
from pyspark.sql.functions import col

# Sắp xếp DataFrame df_date_selected theo thứ tự ưu tiên year, month, day
df = df.orderBy(col("year"), col("month"), col("day"))


In [0]:
df = df.dropna(subset=['year', 'month', 'day'])

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

# Thêm cột ID tăng dần cho DataFrame
df_with_id = df.withColumn("id", monotonically_increasing_id())


In [0]:
from pyspark.sql.functions import monotonically_increasing_id

# Chọn cột "id" và sau đó chọn tất cả các cột còn lại
df_with_id = df_with_id.select('id','day','month','year','date_listed')

df_with_id = df_with_id.withColumnRenamed("id", "id_date")



In [0]:
df_with_id.show()

+-------+---+-----+----+-----------+
|id_date|day|month|year|date_listed|
+-------+---+-----+----+-----------+
|      0| 29|    4|2024| 2024-04-29|
|      1| 30|    4|2024| 2024-04-30|
|      2|  1|    5|2024| 2024-05-01|
|      3|  2|    5|2024| 2024-05-02|
|      4|  3|    5|2024| 2024-05-03|
|      5|  4|    5|2024| 2024-05-04|
|      6|  5|    5|2024| 2024-05-05|
|      7|  6|    5|2024| 2024-05-06|
|      8|  7|    5|2024| 2024-05-07|
|      9|  8|    5|2024| 2024-05-08|
|     10|  9|    5|2024| 2024-05-09|
|     11| 10|    5|2024| 2024-05-10|
|     12| 11|    5|2024| 2024-05-11|
|     13| 12|    5|2024| 2024-05-12|
|     14| 13|    5|2024| 2024-05-13|
+-------+---+-----+----+-----------+



In [0]:
df_with_id.write.format("delta").mode("overwrite").saveAsTable("gold_dimdate")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1738977248255201>, line 1
----> 1 df_with_id.write.format("delta").mode("overwrite").saveAsTable("gold_dimdate")

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1804, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
   1802 if format is not None:
   1803     self.format(format)
-> 1804 self._jwrite.saveAsTable(name)

File /databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py:1355, in JavaMember.__ca